In [1]:
import json
def strip_all_entities(text):
    entity_prefixes = ['@','#',"\\"]
    for separator in  string.punctuation:
        if separator not in entity_prefixes :
            text = text.replace(separator,' ')
    words = []
    for word in text.split():
        word = word.strip()
        if word:
            if word[0] not in entity_prefixes:
                words.append(word)
    return ' '.join(words)


def escapejs(val):
    return json.dumps(str(val))

# Aggregate Analysis

In [2]:
#Main page with combined statistics and visualization for India,Indonesia and Thailand
# 1) Total no of events with event type Riots
# 2) Total no of events with event type Protests
# 3) Total no of events with event type Violence against civilians
# 4) Morris donut for Distribution by Country
# 5) Bar chart for Distribution by Event Types
# 6) Events Timeline
from werkzeug.wrappers import Request, Response
from flask import Flask
import re,string
import tablib
import os
from flask import Flask, flash, redirect, render_template, request, session, abort, url_for
import sqlite3
from flask import g

#change this path as per your local DB location
DATABASE = '/Users/prana/Desktop/air/Shodh/db/AIR_DB.db'


conn = sqlite3.connect(DATABASE)
c = conn.cursor()
jsons = c.execute("""SELECT replace(summary,'"',"") as summary, title, partiesInvolved, location, eventDate, eventType, source FROM agg""").fetchall()
row_headers=[x[0] for x in c.description]

json_data=[]
for result in jsons:
    json_data.append(dict(zip(row_headers,result)))

locCnt = c.execute("""select distinct(location) as location, count(location) as cnt from agg group by location order by cnt desc limit 5""").fetchall()
row_headers=[x[0] for x in c.description]
loc_data=[]
for result in locCnt:
    loc_data.append(dict(zip(row_headers,result)))


srcCnt = c.execute("""select distinct(eventDate) as date, count(source) as cnt from agg where source='India Today' group by date""").fetchall()
row_headers=[x[0] for x in c.description]
src_data=[]
for result in srcCnt:
    src_data.append(dict(zip(row_headers,result)))

parties = c.execute("""select partiesInvolved as label, count(partiesInvolved) as value from agg where not  label ='other' group by label order by value desc limit 5;""").fetchall()
row_headers=[x[0] for x in c.description]
party_data=[]
for result in parties:
    party_data.append(dict(zip(row_headers,result)))

    
event = c.execute("""select eventType, count(eventType) as cnt from agg where not eventType='other' group by eventType order by eventType;""").fetchall()
row_headers=[x[0] for x in c.description]
event_data=[]
for result in event:
    event_data.append(dict(zip(row_headers,result)))

country = c.execute("""select country, count(country) as cnt from agg group by country;""").fetchall()
row_headers=[x[0] for x in c.description]
country_data=[]
for result in country:
    country_data.append(dict(zip(row_headers,result)))

    
series = c.execute("""
SELECT * FROM 
    (SELECT 
    eventDate, 
    SUM(CASE eventType WHEN 'Riots' THEN 1 ELSE 0 END) as Riots,
    SUM(CASE eventType WHEN 'Protests' THEN 1 ELSE 0 END) as Protests,
    SUM(CASE eventType WHEN 'Violence Against Civilians' THEN 1 ELSE 0 END) as ViolenceCivilians 
FROM 
    agg
GROUP BY 
    eventDate
ORDER BY
    eventDate DESC
LIMIT 7) RESULT
ORDER BY eventDate;
""").fetchall()
row_headers=[x[0] for x in c.description]
series_data=[]
for result in series:
    series_data.append(dict(zip(row_headers,result)))
    
    
ticker = c.execute("""
SELECT 
    country, 
    SUM(CASE eventType WHEN 'Riots' THEN 1 ELSE 0 END) as Riots,
    SUM(CASE eventType WHEN 'Protests' THEN 1 ELSE 0 END) as Protests,
    SUM(CASE eventType WHEN 'Violence Against Civilians' THEN 1 ELSE 0 END) as ViolenceCivilians 
FROM 
    agg
GROUP BY 
    country
ORDER BY
    country;
""").fetchall()
row_headers=[x[0] for x in c.description]
ticker_data=[]
for result in ticker:
    ticker_data.append(dict(zip(row_headers,result)))

    
app = Flask(__name__,static_folder='./static')

@app.route("/")
def index():
    return render_template('index.html',data=json_data, locCount = locCnt, sData = src_data, partyData = party_data, eventCount = event_data, seriesData = series_data, countryData = country_data, tickerData=ticker_data)

if __name__ == "__main__":
    app.run()
    

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [26/Jul/2019 16:47:03] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/Jul/2019 16:47:03] "GET /static/css/theme-default.css HTTP/1.1" 200 -
127.0.0.1 - - [26/Jul/2019 16:47:03] "GET /static/js/plugins/bootstrap/bootstrap.min.js HTTP/1.1" 200 -
127.0.0.1 - - [26/Jul/2019 16:47:03] "GET /static/js/plugins/mcustomscrollbar/jquery.mCustomScrollbar.min.js HTTP/1.1" 200 -
127.0.0.1 - - [26/Jul/2019 16:47:03] "GET /static/js/plugins/icheck/icheck.min.js HTTP/1.1" 200 -
127.0.0.1 - - [26/Jul/2019 16:47:03] "GET /static/js/plugins/scrolltotop/scrolltopcontrol.js HTTP/1.1" 200 -
127.0.0.1 - - [26/Jul/2019 16:47:03] "GET /static/js/plugins/morris/raphael-min.js HTTP/1.1" 200 -
127.0.0.1 - - [26/Jul/2019 16:47:03] "GET /static/js/plugins/morris/morris.min.js HTTP/1.1" 200 -
127.0.0.1 - - [26/Jul/2019 16:47:03] "GET /static/js/plugins/rickshaw/d3.v3.js HTTP/1.1" 200 -
127.0.0.1 - - [26/Jul/2019 16:47:03] "GET /static/js/plugins/ric

## INDIA

In [3]:
#This snippet contains code for visualization related to India
# 1) Article title,summary,location,Parties Involved,Event type with pagination
# 2) Tiles for total no of events per eventType, for India
# 3) Filetrs to display the articles for last 24 hours, last 2 days and last week
# 4) Map for displaying the top 5 locations with maximum violence related event count
# 5) Bar chart for top five cities with maximum violence realted activities
# 6) Donut chart of Distribution of Parties Involved

from werkzeug.wrappers import Request, Response
from flask import Flask
import re,string
import tablib
import os
from flask import Flask, flash, redirect, render_template, request, session, abort, url_for
import sqlite3
from flask import g

#change this path as per your local DB location
#DATABASE = '/Users/prana/Desktop/AIR/AIR_DB.db'
DATABASE = '/Users/prana/Desktop/air/Shodh/db/AIR_DB.db'



conn = sqlite3.connect(DATABASE)
c = conn.cursor()
jsons = c.execute("""SELECT replace(summary,'"',"") as summary, title, partiesInvolved, location, eventDate, eventType, source FROM india order by eventDate desc""").fetchall()
row_headers=[x[0] for x in c.description]

json_data=[]
for result in jsons:
    json_data.append(dict(zip(row_headers,result)))

locCnt = c.execute("""select distinct(location) as location, count(location) as cnt from india group by location order by cnt desc limit 5""").fetchall()
row_headers=[x[0] for x in c.description]
loc_data=[]
for result in locCnt:
    loc_data.append(dict(zip(row_headers,result)))


srcCnt = c.execute("""select distinct(eventDate) as date, count(source) as cnt from india where source='India Today' group by date""").fetchall()
row_headers=[x[0] for x in c.description]
src_data=[]
for result in srcCnt:
    src_data.append(dict(zip(row_headers,result)))

parties = c.execute("""select partiesInvolved as label, count(partiesInvolved) as value from india where not  label ='other' group by label order by value desc limit 5;""").fetchall()
row_headers=[x[0] for x in c.description]
party_data=[]
for result in parties:
    party_data.append(dict(zip(row_headers,result)))

    
event = c.execute("""select eventType, count(eventType) as cnt from india where not eventType='other' group by eventType order by eventType;""").fetchall()
row_headers=[x[0] for x in c.description]
event_data=[]
for result in event:
    event_data.append(dict(zip(row_headers,result)))

app = Flask(__name__,static_folder='./static')

@app.route("/")
def index():
    return render_template('india.html',data=json_data, locCount = locCnt, sData = src_data, partyData = party_data, eventCount = event_data)

@app.route("/india.html")
def india2():
    return render_template('india.html')

if __name__ == "__main__":
    app.run()
    

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [26/Jul/2019 16:48:49] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/Jul/2019 16:48:49] "GET /static/css/pagination-css.css HTTP/1.1" 200 -
127.0.0.1 - - [26/Jul/2019 16:48:49] "GET /static/js/plugins/jvectormap/jquery-jvectormap-asia-mill.js HTTP/1.1" 200 -
127.0.0.1 - - [26/Jul/2019 16:48:49] "GET /static/js/pagination.js HTTP/1.1" 200 -
127.0.0.1 - - [26/Jul/2019 16:48:50] "GET /img/backgrounds/wall_1.jpg HTTP/1.1" 404 -
127.0.0.1 - - [26/Jul/2019 16:48:50] "GET /img/backgrounds/wall_2.jpg HTTP/1.1" 404 -
127.0.0.1 - - [26/Jul/2019 16:48:50] "GET /img/backgrounds/wall_3.jpg HTTP/1.1" 404 -
127.0.0.1 - - [26/Jul/2019 16:48:50] "GET /img/backgrounds/wall_4.jpg HTTP/1.1" 404 -
127.0.0.1 - - [26/Jul/2019 16:48:50] "GET /img/backgrounds/wall_5.jpg HTTP/1.1" 404 -
127.0.0.1 - - [26/Jul/2019 16:48:50] "GET /img/backgrounds/wall_6.jpg HTTP/1.1" 404 -
127.0.0.1 - - [26/Jul/2019 16:48:50] "GET /img/backgrounds/wall_7.jpg HTT

# Thailand

In [65]:
#This snippet contains code for visualization related to Thailand
# 1) Article title,summary,location,Parties Involved,Event type with pagination
# 2) Tiles for total no of events per eventType, for Thailand
# 3) Filetrs to display the articles for last 24 hours, last 2 days and last week
# 4) Map for displaying the top 5 locations with maximum violence related event count
# 5) Bar chart for top five cities with maximum violence realted activities
# 6) Donut chart of Distribution of Parties Involved


from werkzeug.wrappers import Request, Response
from flask import Flask
import re,string
import tablib
import os
from flask import Flask, flash, redirect, render_template, request, session, abort, url_for
import sqlite3
from flask import g

#change this path as per your local DB location
DATABASE = '/Users/prana/Desktop/AIR Final/db/AIR_DB.db'


conn = sqlite3.connect(DATABASE)
c = conn.cursor()
jsons = c.execute("""SELECT replace(summary,'"',"") as summary, title, partiesInvolved, location, eventDate, eventType, source FROM thailand order by eventDate desc""").fetchall()

row_headers=[x[0] for x in c.description]

json_data=[]
for result in jsons:
    json_data.append(dict(zip(row_headers,result)))

locCnt = c.execute("""select distinct(location) as location, count(location) as cnt from thailand group by location order by cnt desc limit 5""").fetchall()
row_headers=[x[0] for x in c.description]
loc_data=[]
for result in locCnt:
    loc_data.append(dict(zip(row_headers,result)))


srcCnt = c.execute("""select distinct(eventDate) as date, count(source) as cnt from thailand where source='India Today' group by date""").fetchall()
row_headers=[x[0] for x in c.description]
src_data=[]
for result in srcCnt:
    src_data.append(dict(zip(row_headers,result)))

parties = c.execute("""select partiesInvolved as label, count(partiesInvolved) as value from thailand where not  label ='other' group by label order by value desc limit 5;""").fetchall()
row_headers=[x[0] for x in c.description]
party_data=[]
for result in parties:
    party_data.append(dict(zip(row_headers,result)))

    
event = c.execute("""select eventType, count(eventType) as cnt from thailand where not eventType='other' group by eventType order by eventType;""").fetchall()
row_headers=[x[0] for x in c.description]
event_data=[]
for result in event:
    event_data.append(dict(zip(row_headers,result)))


app = Flask(__name__,static_folder='./static')

@app.route("/")
def index():
    return render_template('thailand.html',data=json_data, locCount = locCnt, sData = src_data, partyData = party_data, eventCount = event_data)

@app.route("/india")
def india1():
    return render_template('india.html')

if __name__ == "__main__":
    app.run()
    

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [06/May/2019 18:26:39] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [06/May/2019 18:26:40] "GET /img/backgrounds/wall_1.jpg HTTP/1.1" 404 -
127.0.0.1 - - [06/May/2019 18:26:40] "GET /img/backgrounds/wall_2.jpg HTTP/1.1" 404 -
127.0.0.1 - - [06/May/2019 18:26:40] "GET /img/backgrounds/wall_3.jpg HTTP/1.1" 404 -
127.0.0.1 - - [06/May/2019 18:26:40] "GET /img/backgrounds/wall_4.jpg HTTP/1.1" 404 -
127.0.0.1 - - [06/May/2019 18:26:40] "GET /img/backgrounds/wall_5.jpg HTTP/1.1" 404 -
127.0.0.1 - - [06/May/2019 18:26:40] "GET /img/backgrounds/wall_6.jpg HTTP/1.1" 404 -
127.0.0.1 - - [06/May/2019 18:26:40] "GET /img/backgrounds/wall_7.jpg HTTP/1.1" 404 -
127.0.0.1 - - [06/May/2019 18:26:40] "GET /img/backgrounds/wall_8.jpg HTTP/1.1" 404 -
127.0.0.1 - - [06/May/2019 18:26:40] "GET /img/backgrounds/wall_9.jpg HTTP/1.1" 404 -
127.0.0.1 - - [06/May/2019 18:26:40] "GET /img/backgrounds/wall_10.jpg HTTP/1.1" 404 -


# Indonesia

In [2]:
#This snippet contains code for visualization related to Indonesia
# 1) Article title,summary,location,Parties Involved,Event type with pagination
# 2) Tiles for total no of events per eventType, for Indonesia
# 3) Filetrs to display the articles for last 24 hours, last 2 days and last week
# 4) Map for displaying the top 5 locations with maximum violence related event count
# 5) Bar chart for top five cities with maximum violence realted activities
# 6) Donut chart of Distribution of Parties Involved

from werkzeug.wrappers import Request, Response
from flask import Flask
import re,string
import tablib
import os
from flask import Flask, flash, redirect, render_template, request, session, abort, url_for
import sqlite3
from flask import g

#change this path as per your local DB location
DATABASE = '/Users/prana/Desktop/AIR Final/db/AIR_DB.db'

conn = sqlite3.connect(DATABASE)
c = conn.cursor()
jsons = c.execute("""SELECT replace(summary,'"',"") as summary, title, partiesInvolved, location, eventDate, eventType, source FROM indonesia order by eventDate desc""").fetchall()
row_headers=[x[0] for x in c.description]

json_data=[]
for result in jsons:
    json_data.append(dict(zip(row_headers,result)))

locCnt = c.execute("""select distinct(location) as location, count(location) as cnt from indonesia group by location order by cnt desc limit 5""").fetchall()
row_headers=[x[0] for x in c.description]
loc_data=[]
for result in locCnt:
    loc_data.append(dict(zip(row_headers,result)))

srcCnt = c.execute("""select distinct(eventDate) as date, count(source) as cnt from indonesia where source='India Today' group by date""").fetchall()
row_headers=[x[0] for x in c.description]
src_data=[]
for result in srcCnt:
    src_data.append(dict(zip(row_headers,result)))

parties = c.execute("""select partiesInvolved as label, count(partiesInvolved) as value from indonesia where not  label ='other' group by label order by value desc limit 5;""").fetchall()
row_headers=[x[0] for x in c.description]
party_data=[]
for result in parties:
    party_data.append(dict(zip(row_headers,result)))

    
event = c.execute("""select eventType, count(eventType) as cnt from indonesia where not eventType='other' group by eventType order by eventType;""").fetchall()
row_headers=[x[0] for x in c.description]
event_data=[]
for result in event:
    event_data.append(dict(zip(row_headers,result)))


app = Flask(__name__,static_folder='./static')

@app.route("/")
def index():
    return render_template('indonesia.html',data=json_data, locCount = locCnt, sData = src_data, partyData = party_data, eventCount = event_data)

if __name__ == "__main__":
    app.run()
    

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [10/May/2019 19:21:01] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2019 19:21:01] "GET /static/css/pagination-css.css HTTP/1.1" 304 -
127.0.0.1 - - [10/May/2019 19:21:01] "GET /static/js/plugins/jvectormap/jquery-jvectormap-asia-mill.js HTTP/1.1" 304 -
127.0.0.1 - - [10/May/2019 19:21:01] "GET /static/js/pagination.js HTTP/1.1" 304 -
127.0.0.1 - - [10/May/2019 19:21:01] "GET /img/backgrounds/wall_1.jpg HTTP/1.1" 404 -
127.0.0.1 - - [10/May/2019 19:21:01] "GET /img/backgrounds/wall_2.jpg HTTP/1.1" 404 -
127.0.0.1 - - [10/May/2019 19:21:01] "GET /img/backgrounds/wall_3.jpg HTTP/1.1" 404 -
127.0.0.1 - - [10/May/2019 19:21:01] "GET /img/backgrounds/wall_4.jpg HTTP/1.1" 404 -
127.0.0.1 - - [10/May/2019 19:21:01] "GET /img/backgrounds/wall_5.jpg HTTP/1.1" 404 -
127.0.0.1 - - [10/May/2019 19:21:01] "GET /img/backgrounds/wall_7.jpg HTTP/1.1" 404 -
127.0.0.1 - - [10/May/2019 19:21:01] "GET /img/backgrounds/wall_6.jpg HTT

In [63]:
c.close()